## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.



Suggestion: follow the below plan of action:

* Get familiar with the data
* Select columns of interest
* Decide what to do with NaNs

* Make processing plan
* Develop code that works with a sample

* Adjust the code to work with Big data
* Test big data approach on a sample

* Run program with big data

You can skip the first step this time, since you already did it for the other exercises

## 2) Prepare the data for processing

### Booking

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
bookings_path = '../../../data/Challenge/bookings.csv.bz2'
searches_path = '../../../data/Challenge/searches.csv.bz2'

bookings_small_sample = pd.read_csv(bookings_path, sep='^', nrows=100)
searches_small_sample = pd.read_csv(searches_path, sep='^', nrows=100)

We take small samples in order to determine column types. This is a bit risky because it can fail if the types don't fit the rest of the columns but it lets us save some memory, because pandas won't scan the whole file to determine types.

In [3]:
bookings_types = bookings_small_sample.dtypes.to_dict()

bookings_sample = pd.read_csv(bookings_path, sep='^', nrows=10000, dtype=bookings_types)

In [4]:
bookings_sample.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

In [5]:
cols_to_use_bookings = ['cre_date           ', 'dep_port', 'arr_port']

In [6]:
searches_small_sample.columns

Index(['Date', 'Time', 'TxnCode', 'OfficeID', 'Country', 'Origin',
       'Destination', 'RoundTrip', 'NbSegments', 'Seg1Departure',
       'Seg1Arrival', 'Seg1Date', 'Seg1Carrier', 'Seg1BookingCode',
       'Seg2Departure', 'Seg2Arrival', 'Seg2Date', 'Seg2Carrier',
       'Seg2BookingCode', 'Seg3Departure', 'Seg3Arrival', 'Seg3Date',
       'Seg3Carrier', 'Seg3BookingCode', 'Seg4Departure', 'Seg4Arrival',
       'Seg4Date', 'Seg4Carrier', 'Seg4BookingCode', 'Seg5Departure',
       'Seg5Arrival', 'Seg5Date', 'Seg5Carrier', 'Seg5BookingCode',
       'Seg6Departure', 'Seg6Arrival', 'Seg6Date', 'Seg6Carrier',
       'Seg6BookingCode', 'From', 'IsPublishedForNeg', 'IsFromInternet',
       'IsFromVista', 'TerminalID', 'InternetOffice'],
      dtype='object')

It works fine with the bookings file, but not with the searches file. That's because there are some empty columns in the first few records that have strings in later records. Pandas will infer a `float64` type when it sees an empty column.

http://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options

In [7]:
bookings_sample = pd.read_csv(bookings_path, 
                              sep='^', 
                              nrows=10000, 
                              dtype=bookings_small_sample.dtypes.to_dict(),
                              usecols=cols_to_use_bookings)


searches_sample = pd.read_csv(searches_path, 
                              sep='^',
                              dtype=searches_small_sample.dtypes.to_dict(),
                              nrows=10000)

ValueError: could not convert string to float: 'JNB'

In [8]:
searches_small_sample.dtypes

Date                  object
Time                  object
TxnCode               object
OfficeID              object
Country               object
Origin                object
Destination           object
RoundTrip              int64
NbSegments             int64
Seg1Departure         object
Seg1Arrival           object
Seg1Date              object
Seg1Carrier           object
Seg1BookingCode       object
Seg2Departure         object
Seg2Arrival           object
Seg2Date              object
Seg2Carrier           object
Seg2BookingCode       object
Seg3Departure         object
Seg3Arrival           object
Seg3Date              object
Seg3Carrier           object
Seg3BookingCode       object
Seg4Departure         object
Seg4Arrival           object
Seg4Date              object
Seg4Carrier           object
Seg4BookingCode       object
Seg5Departure        float64
Seg5Arrival          float64
Seg5Date             float64
Seg5Carrier          float64
Seg5BookingCode      float64
Seg6Departure 

In [9]:
bookings_sample = pd.read_csv(bookings_path, 
                              sep='^', 
                              nrows=10000, 
                              dtype=bookings_small_sample.dtypes.to_dict(),
                              usecols=cols_to_use_bookings)


searches_sample = pd.read_csv(searches_path, 
                              sep='^',
                              nrows=10000)

In [10]:
searches_sample.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0,1,RNB,...,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1,2,OSL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL


#### We didnt check for duplicates so far... What if the file is has duplicated lines?

The bookings file can be reduced a lot for our purpose here. There are not that many combinations of origin, destination, and creation date.

In [11]:
bookings = pd.read_csv(bookings_path, 
                              sep='^', 
                              dtype=bookings_small_sample.dtypes.to_dict(),
                              usecols=cols_to_use_bookings)

deduped_bookings = bookings.drop_duplicates()
deduped_bookings.shape

(334877, 3)

#### We have seen that we have white space in some columns....

Careful with this kind of thing!!

In [12]:
deduped_bookings.columns = deduped_bookings.columns.str.strip()
deduped_bookings.columns

Index(['cre_date', 'dep_port', 'arr_port'], dtype='object')

In [13]:
deduped_bookings['arr_port'].unique()

array(['LHR     ', 'CLT     ', 'SVO     ', ..., 'VEL     ', 'SLM     ',
       'SG      '], dtype=object)

#### Could we do this with command line?

## 3) Make processing plan

Target: generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.

1) remove duplicates

2) remove whitespaces

    a) from colum names
    
    b) from content

4) remove NaN

5) define the model
    if there is one booking for a given O&D done at the same day as the search (for a given O&D), ALL searches of the day (for a given 0&D) might have resulted from the same source and will be set with 1.
    This is regardless of the boarding time of the plane... So if I was looking for plane for the first 4 days of December for a given O&D all those searches would be set to 1 not just the one correspoding to the correct boarding time

        match
        Search : [search_date, O&D] 
        Booking: [Activity_date, O&D]

6) execute the model

    a) Dedup bookings on [Activity_date, O&D] so that we dont have duplicates (and we can have number of bookings for the day), or we can just drop the duplicates
    
    b) search left join bookings adding "Booked" column
    
    c) test if the merge was done right
    
    d) fill NaN of "booked" column with 0
    
    e) pull all values of booked column >1 to 1

What do we have?

In [14]:
deduped_bookings.head()

,cre_date,dep_port,arr_port
0,2013-02-22 00:00:00,ZRH,LHR
1,2013-03-26 00:00:00,SAL,CLT
3,2013-03-26 00:00:00,AKL,SVO
5,2013-03-20 00:00:00,DEN,LGA
7,2013-03-25 00:00:00,NRT,SIN


In [15]:
searches_sample.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0,1,RNB,...,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1,2,OSL,...,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL


We can't match-> need to format. We use, once again, the .str attribute of the column of  interest.

In [16]:
deduped_bookings['cre_date'] = deduped_bookings['cre_date'].str[:10]
deduped_bookings.head()

/home/dani/anaconda3/envs/master-p3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,cre_date,dep_port,arr_port
0,2013-02-22,ZRH,LHR
1,2013-03-26,SAL,CLT
3,2013-03-26,AKL,SVO
5,2013-03-20,DEN,LGA
7,2013-03-25,NRT,SIN


Let's clean the whitespace we saw earlier...

In [17]:
deduped_bookings['dep_port'] = deduped_bookings['dep_port'].str.strip()
deduped_bookings['arr_port'] = deduped_bookings['arr_port'].str.strip()
deduped_bookings.iloc[0]['dep_port']

/home/dani/anaconda3/envs/master-p3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/dani/anaconda3/envs/master-p3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


'ZRH'

... and check whether we need to in the searches file:

In [18]:
lens = searches_sample['Origin'].apply(len)
lens.unique()

array([3])

In [19]:
# Another way to check:

lens = searches_sample['Origin'].apply(len)
np.all(lens == 3)

True

In [20]:
lens = searches_sample['Destination'].apply(len)
np.all(lens == 3)

True

We will need to put a 'booked' column in the result after the join. We might as well create it now.

In [ ]:
deduped_bookings['booked'] = 1
deduped_bookings.head()

We do the join...

In [22]:
joined = searches_sample.merge(deduped_bookings, 
                               left_on=['Origin', 'Destination', 'Date'], 
                               right_on=['dep_port', 'arr_port', 'cre_date'],
                               how = 'left')

joined.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,cre_date,dep_port,arr_port,booked
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1,2,TXL,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,NaN,NaN,NaN,NaN
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0,1,ATH,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,NaN,NaN,NaN,NaN
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1,2,ICT,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,NaN,NaN,NaN,NaN
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0,1,RNB,...,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,NaN,NaN,NaN,NaN
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1,2,OSL,...,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,NaN,NaN,NaN,NaN


Drop the extra columns we got from the bookings file, which we don't care about, and fill the NaNs (not booked) with 0s, as per the wording in the problem statement.

In [23]:
searches_matched = joined.drop(['cre_date', 'dep_port', 'arr_port'], axis=1)

searches_matched['booked'][searches_matched['booked'].isnull()] = 0

/home/dani/anaconda3/envs/master-p3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
searches_matched['booked'] = searches_matched['booked'].fillna(0)

#### Put in a single cell the whole processing

From the input files, with a sample of searches:

In [25]:
# Input variables
bookings_path = '../../../data/Challenge/bookings.csv.bz2'
searches_path = '../../../data/Challenge/searches.csv.bz2'

# Check types
bookings_small_sample = pd.read_csv(bookings_path, sep='^', nrows=100)
bookings_types = bookings_small_sample.dtypes.to_dict()

# Read in the bookings file
cols_to_use_bookings = ['cre_date           ', 'dep_port', 'arr_port']
bookings = pd.read_csv(bookings_path, 
                              sep='^', 
                              dtype=bookings_small_sample.dtypes.to_dict(),
                              usecols=cols_to_use_bookings)

# Clean up the dataframe
deduped_bookings = bookings.drop_duplicates()
deduped_bookings.columns = deduped_bookings.columns.str.strip()
deduped_bookings['dep_port'] = deduped_bookings['dep_port'].str.strip()
deduped_bookings['arr_port'] = deduped_bookings['arr_port'].str.strip()
deduped_bookings['cre_date'] = deduped_bookings['cre_date'].str[:10]

# Introduce dummy column
deduped_bookings['booked'] = 1


# Read searches file
searches_sample = pd.read_csv(searches_path, 
                              sep='^',
                              nrows=10000)

# Join
joined = searches_sample.merge(deduped_bookings, 
                               left_on=['Origin', 'Destination', 'Date'], 
                               right_on=['dep_port', 'arr_port', 'cre_date'],
                               how = 'left')

# Clean up the resulting dataframe
searches_matched = joined.drop(['cre_date', 'dep_port', 'arr_port'], axis=1)
searches_matched['booked'] = searches_matched['booked'].fillna(0)

/home/dani/anaconda3/envs/master-p3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dani/anaconda3/envs/master-p3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dani/anaconda3/envs/master-p3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

### Adapt it to Big Data

We can hold the reduced bookings file in memory, so we'll process it in one go: this saves us a LOT of processing time. 

The searches file however needs to be processed in chunks. In this case we are not going to reduce it, since we are required to write it back to disk. However, we can safely process it without using a lot of memory by _writing to disk each chunk as we generate it_. This way we don't need to hold more than one chunk in memory at a time.


In [26]:
# Input variables
bookings_path = '../../../data/Challenge/bookings.csv.bz2'
searches_path = '../../../data/Challenge/searches.csv.bz2'
result_path = './searches.matched.csv.bz2'

# Check types
bookings_small_sample = pd.read_csv(bookings_path, sep='^', nrows=100)
bookings_types = bookings_small_sample.dtypes.to_dict()

# Read in the bookings file
cols_to_use_bookings = ['cre_date           ', 'dep_port', 'arr_port']
bookings = pd.read_csv(bookings_path, 
                              sep='^', 
                              dtype=bookings_small_sample.dtypes.to_dict(),
                              usecols=cols_to_use_bookings)

# Clean up the dataframe
deduped_bookings = bookings.drop_duplicates()
deduped_bookings.columns = deduped_bookings.columns.str.strip()
deduped_bookings['dep_port'] = deduped_bookings['dep_port'].str.strip()
deduped_bookings['arr_port'] = deduped_bookings['arr_port'].str.strip()
deduped_bookings['cre_date'] = deduped_bookings['cre_date'].str[:10]

# Introduce dummy column
deduped_bookings['booked'] = 1


# Read searches file
chunks = pd.read_csv(searches_path, 
                              sep='^',
                              chunksize=1000000)

chunk_num = 0 

for chunk in chunks:
    print(chunk_num)
    
    # Join
    joined = chunk.merge(deduped_bookings, 
                           left_on=['Origin', 'Destination', 'Date'], 
                           right_on=['dep_port', 'arr_port', 'cre_date'],
                           how = 'left')

    # Clean up the resulting dataframe
    searches_matched = joined.drop(['cre_date', 'dep_port', 'arr_port'], axis=1)
    searches_matched['booked'] = searches_matched['booked'].fillna(0)
    
    # And write it to disk, using mode 'append' in order not to
    # overwrite the result of the previous chunks.
    # We need to make sure we only write the header once, using
    # the header argument to to_csv.
    searches_matched.to_csv(result_path, 
                            sep ='^', 
                            compression='bz2', 
                            mode='a', 
                            header=chunk_num==0,
                            index=False)
    
    chunk_num+=1


/home/dani/anaconda3/envs/master-p3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dani/anaconda3/envs/master-p3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/dani/anaconda3/envs/master-p3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

0


/home/dani/anaconda3/envs/master-p3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (40,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1
2


KeyboardInterrupt: 